In [1]:
import numpy as np

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
# %matplotlib tk
we_keys = [0, 10, -1]
eff_keys = [0, 0.2, 0.6, 1]
db_keys = [0, 1, 10, 100]


In [3]:
import pandas as pd

# Dataframe from csv
df = pd.read_csv("EGG_DATA/report.csv")
df["dir"] = df["dir"].apply(lambda x: int(x))
# Time to survival -1 to inf
for i in range(5):
    df[f"time_to_survival_agent_{i}"] = df[f"time_to_survival_agent_{i}"].apply(lambda x: np.infty if x == -1 else x)
    df[f"time_to_survival_agent_sd{i}"] = df[f"time_to_survival_agent_sd{i}"].apply(lambda x: np.infty if x == -1 else x)

df.info()

FileNotFoundError: [Errno 2] No such file or directory: 'EGG_DATA/report.csv'

In [ ]:
# Pretty print table with eff, db, we and dir
df[["eff_rate", "db", "we", "dir"]]

In [ ]:
df_numpy = df.to_numpy()

# Reference policies: we = 10
ref_policies_values = {
    eff: {db: np.zeros((5,2)) for db in db_keys} for eff in eff_keys
}
ref_policies_np = df_numpy[df_numpy[:, 2] == 10]

for i in range(len(ref_policies_np)):
    eff = ref_policies_np[i][1]
    db = ref_policies_np[i][0]
    ref_policies_values[eff][db] = np.hstack([ref_policies_np[i][10:15].reshape((5,1)), ref_policies_np[i][15:20].reshape((5,1)) ])

ref_policies_values[0.6][1] = np.array([[-1.3750e+02,  7.3400e+00],[-1.3025e+02,  5.2600e+00],[-1.3400e+02,  7.1700e+00],[-2.5138e+02,  1.0000e-02],[-2.4075e+02,  3.0000e-02]])   

ref_policies_values[0.2][1]

In [ ]:
ref_policies_values[0.6][1]

In [ ]:
unet_policies_values = {
    eff: np.zeros((5,2)) for eff in eff_keys
}

unet_policies_np = df_numpy[df_numpy[:, 2] == 0]
for i in range(len(unet_policies_np)):
    eff = unet_policies_np[i][1]
    unet_policies_values[eff] = np.hstack([unet_policies_np[i][10:15].reshape((5,1)), unet_policies_np[i][15:20].reshape((5,1)) ])
unet_policies_values[0] = ref_policies_values[0][0]
unet_policies_values[1] = ref_policies_values[1][0]
unet_policies_values[0.6]

In [ ]:
# Function to get the intersection of two lines represented as f(we) = v0 + we*ve for a two given fixed sets of (v0, ve)
def get_intersection(v01, ve1, v02, ve2):
    if ve1 == ve2:
        return 0
    we = (v01 - v02) / (ve2 - ve1)
    return we
# get_intersection(mo_value_w10[2][0], mo_value_w10[2][1], mo_value_w0[2][0], mo_value_w0[2][1])

In [ ]:
# Plot values as function of weight w that scales the reward
fig, axs = plt.subplots(1, 5, figsize=(25, 5)) 
step = 0.1
# for ag in mo_value_w10:
max_weight = 10
# Subplot for each agent
x = np.arange(0, max_weight, step)
w = np.array([np.ones(int(max_weight/step)), np.arange(0, max_weight, step)])

setting = {"eff": 0.6, "db": 1}
for ag in range(5):
    axs[ag].set_title("MO Value Function")
    ref = ref_policies_values[setting["eff"]][setting["db"]]
    unet = unet_policies_values[setting["eff"]]
    axs[ag].plot(x, np.dot(unet[ag], w), c="blue")
    axs[ag].plot(x, np.dot(ref[ag], w), c="green")
    
wes = []
for ag in range(5):
    # Print intersecctions
    p1 = unet_policies_values[setting["eff"]][ag]
    p2 = ref_policies_values[setting["eff"]][setting["db"]][ag]
    inter = get_intersection(p1[0], p1[1], p2[0], p2[1])
    print(inter)
    wes.append(inter)
plt.show()

In [ ]:
print(ref_policies_values[0.6][0][0:3])
print(ref_policies_values[0.6][1][0:3])
print(ref_policies_values[0.6][10][0:3])
print(ref_policies_values[0.6][100][0:3])

In [ ]:
def compute_wes_prima(eff, db):
    def aggregation_function(wes, method="median"):
        # cut to interval 1-10 else return -inf
        wes = [w for w in wes if w >= 1 and w <= 10]
        if len(wes) == 0:
            return -np.infty
        if method == "median":
            return np.median(wes)
        if method == "mean":
            return np.mean(wes)
        if method == "max":
            return np.max(wes)
        if method == "min":
            return np.min(wes)
    wes_prima = []
    for ag in range(5):
        # Print intersecctions
        p1 = unet_policies_values[eff][ag]
        p2 = ref_policies_values[eff][db][ag]
        inter = get_intersection(p1[0], p1[1], p2[0], p2[1])
        wes_prima.append(inter)
    return aggregation_function(wes_prima, method="median")

# Create table of WES' for each eff (from 0.2, 0.6, 1) and db (from 0, 1, 10, 100) 
wes_prima = {
    db: {eff: compute_wes_prima(eff, db) for eff in eff_keys} for db in db_keys
}
# pretty print as dataframe
pd.DataFrame(wes_prima)

In [ ]:
time2survival_np = ref_policies_np[:, 20:25]
time2survival_std_np = ref_policies_np[:, 25:30]
# Create a string with the mean and std of the time to survival for each agent
mean_std_time2survival = {(eff, db): [] for eff in eff_keys for db in db_keys}
for i in range(len(time2survival_np)):
    eff = ref_policies_np[i][1]
    db = ref_policies_np[i][0]
    for ag in range(5):
        mean_std_time2survival[eff, db].append(f"{time2survival_np[i][ag]:.2f} ± {time2survival_std_np[i][ag]:.2f}")
# pretty print as dataframe
mean_std_time2survival[0, 10]

In [ ]:
# Print table of the gini
max_gini_value = 1 - 1/5
gini = {
    db: {eff: df[(df['we'] == 10) & (df['eff_rate'] == eff) & (df['db'] == db)]["gini"].to_numpy()/max_gini_value for eff in eff_keys} for db in db_keys
}
pd.DataFrame(gini)

In [ ]:
# Print table of the hoover
max_hoover_value = 1
hoover = {
    db: {eff: df[(df['we'] == 10) & (df['eff_rate'] == eff) & (df['db'] == db)]["hoover"].to_numpy() for eff in eff_keys} for db in db_keys
}
pd.DataFrame(hoover).to_numpy()

In [ ]:
import numpy as np
from tabulate import tabulate

gini = np.array([[np.array([], dtype=np.float64), np.array([0.18]), np.array([0.18]),
        np.array([0.2])],
       [np.array([0.48]), np.array([0.15]), np.array([0.02]), np.array([0.05])],
       [np.array([0.35]), np.array([0.2]), np.array([0.19]), np.array([0.])],
       [np.array([0.11]), np.array([0.18]), np.array([0.1]), np.array([0.])]],
      dtype=object)

def numpy_to_latex(array):    
    latex_table = tabulate(array, tablefmt="latex_raw", headers=db_keys, showindex=eff_keys)
    return latex_table

print(numpy_to_latex(gini))